# Tracing Basics

### Setup

Make sure you set your environment variables, including your OpenAI API key.

In [ ]:
!pip install --quiet -U langchain-google-genai langgraph langgraph-sdk langgraph-checkpoint-sqlite langsmith langchain-community langchain-core
!pip install --quiet notebook python-dotenv lxml scikit-learn pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.6/151.6 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/21

In [ ]:
!pip install --quiet python-dotenv

In [ ]:
from dotenv import load_dotenv
load_dotenv(".env")

True

In [ ]:
# You can set them inline
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [ ]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

### Tracing with @traceable

The @traceable decorator is a simple way to log traces from the LangSmith Python SDK. Simply decorate any function with @traceable.

The decorator works by creating a run tree for you each time the function is called and inserting it within the current trace. The function inputs, name, and other information is then streamed to LangSmith. If the function raises an error or if it returns a response, that information is also added to the tree, and updates are patched to LangSmith so you can detect and diagnose sources of errors. This is all done on a background thread to avoid blocking your app's execution.

In [ ]:
# TODO: Import traceable
from langsmith import traceable

from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

# MODEL_PROVIDER = "openai"
# MODEL_NAME = "gpt-4o-mini"
MODEL_NAME = "gemini-2.5-flash"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the latest question in the conversation.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
"""

import os
from google import genai
# openai_client = OpenAI()
client = genai.Client(api_key=os.getenv('GOOGLE_API_KEY'))
nest_asyncio.apply()
retriever = get_vector_db_retriever()

# TODO: Set up tracing for each function
@traceable
def retrieve_documents(question: str):
    return retriever.invoke(question)   # NOTE: This is a LangChain vector db retriever, so this .invoke() call will be traced automatically

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    # messages = [
    #     {
    #         "role": "system",
    #         "content": RAG_SYSTEM_PROMPT
    #     },
    #     {
    #         "role": "user",
    #         "content": f"Context: {formatted_docs} \n\n Question: {question}"
    #     }
    # ]
    messages = [
                  {
                      "role": "user",
                      "parts": [
                          {"text": RAG_SYSTEM_PROMPT},
                          {"text": f"Context: {formatted_docs} \n\n Question: {question}"}
                      ]
                  }
              ]
    return call_gemini(messages)

@traceable
def call_gemini(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    # return openai_client.chat.completions.create(
    #     model=model,
    #     messages=messages,
    #     temperature=temperature,
    # )
    return client.models.generate_content(
    model=model, contents=messages
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    # return response.choices[0].message.content
    return response.candidates[0].content.parts[0].text


Loading existing vector store from: /tmp/union_local.parquet


@traceable handles the RunTree lifecycle for you!

In [ ]:
question = "How can I trace with the @traceable decorator?"
ai_answer = langsmith_rag(question)
print(ai_answer)

You can use the `@traceable` decorator by applying it directly to a function you want to trace. This allows you to automatically log traces for that function's execution. For example, you can specify a `run_type` and add `metadata` such as provider or model names to the trace.


##### Let's take a look in LangSmith!

### Adding Metadata

LangSmith supports sending arbitrary metadata along with traces.

Metadata is a collection of key-value pairs that can be attached to runs. Metadata can be used to store additional information about a run, such as the version of the application that generated the run, the environment in which the run was generated, or any other information that you want to associate with a run. Similar to tags, you can use metadata to filter runs in the LangSmith UI, and can be used to group runs together for analysis.

In [ ]:
from langsmith import traceable
MODEL_PROVIDER = "google"
@traceable(
    # TODO: Add Metadata
    metadata={"vectordb": "sklearn"}
)
def retrieve_documents(question: str):
    return retriever.invoke(question)

@traceable
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    # messages = [
    #     {
    #         "role": "system",
    #         "content": RAG_SYSTEM_PROMPT
    #     },
    #     {
    #         "role": "user",
    #         "content": f"Context: {formatted_docs} \n\n Question: {question}"
    #     }
    # ]
    messages = [
                  {
                      "role": "user",
                      "parts": [
                          {"text": RAG_SYSTEM_PROMPT},
                          {"text": f"Context: {formatted_docs} \n\n Question: {question}"}
                      ]
                  }
              ]
    return call_gemini(messages)

@traceable(
    # TODO: Add Metadata
    metadata={"model_name": MODEL_NAME, "model_provider": MODEL_PROVIDER}
)
def call_gemini(
    messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0
) -> str:
    # return openai_client.chat.completions.create(
    #     model=model,
    #     messages=messages,
    #     temperature=temperature,
    # )
    return client.models.generate_content(
    model=model, contents=messages
    )

@traceable
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    # return response.choices[0].message.content
    return response.candidates[0].content.parts[0].text



In [ ]:
question = "How do I add Metadata to a Run with @traceable?"
ai_answer = langsmith_rag(question)
print(ai_answer)

You can add metadata to a run with the `@traceable` decorator by passing a dictionary to the `metadata` argument. For example, you can include `metadata={"ls_provider": "my_provider", "ls_model_name": "my_model"}` directly within the decorator. This allows you to store additional key-value pairs associated with the run.


You can also add metadata at runtime!

In [ ]:
question = "How do I add metadata at runtime?"
ai_answer = langsmith_rag(question, langsmith_extra={"metadata": {"runtime_metadata": "foo"}})
print(ai_answer)

You can add metadata at runtime by first getting the current run tree. Then, you can set metadata on the parent run using `rt = ls.get_current_run_tree()` and `rt.metadata["your-key"] = "your-value"`. You can also extend tags dynamically using `rt.tags.extend(["another-tag"])`.


##### Let's take a look in LangSmith!